In [ ]:
# default_exp data
# default_cls_lvl 3

In [ ]:
#export
import pandas as pd
import numpy as np

from task_substitution.core import *

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

### Dataset Class

In [ ]:
#export
class Dataset:
    def __init__(self, df, **kwargs):
        self.df = df
        self.missing_fld = kwargs['missing_fld']
        self.ignore_flds = kwargs['ignore_flds']
        self.cat_flds = kwargs['cat_flds']
        
    @property
    def target(self):
        return self.data[self.missing_fld]
    
    def remove_ignore_flds(self)->pd.DataFrame:
        if self.ignore_flds is not None:
            self.df = _ignore_flds(self.df, self.ignore_flds)
        return self.df
    
    def preprocess_categorical(self)->pd.DataFrame:
        cats = self.cat_flds if self.cat_flds else self.df.select_dtypes(include=['object']).columns
        for c, v in self.df.loc[:, cats].items():
            self.df.loc[:, c] = _preprocess_categorical(v)
        return self.df
    
    def preprocess(self):
        self.df = self.remove_ignore_flds()
        self.df = self.preprocess_categorical()
        
        return self.df
    
    @classmethod
    def split_train_test(cls, df, missing_fld):
        """
        Splits the dataframe into train and test dataset based on the rows having values
        for missing feature or not.
        """
        train, test = _split_train_test(df, missing_fld)
        return train, test

In [ ]:
import string
example_df = pd.DataFrame({'c1': np.random.rand(10, ),
                           'c2': [string.ascii_lowercase[np.random.randint(low=0, high=10)] for i in range(10)],
                           'c3': np.random.permutation([np.nan] + list(np.random.rand(9, )))
                          })
example_df

,c1,c2,c3
0,0.029411,d,0.305660
1,0.217001,f,0.883794
2,0.128505,h,NaN
3,0.263810,c,0.104287
4,0.150202,f,0.957786
5,0.464820,e,0.311979
6,0.792108,d,0.766451
7,0.552352,e,0.891954
8,0.164857,i,0.143009
9,0.528950,j,0.497675


In [ ]:
data = Dataset(example_df, missing_fld='c3', cat_flds=['c2'], ignore_flds=None)
new_example_df = data.preprocess(); new_example_df

,c1,c2,c3
0,0.029411,2,0.305660
1,0.217001,4,0.883794
2,0.128505,5,NaN
3,0.263810,1,0.104287
4,0.150202,4,0.957786
5,0.464820,3,0.311979
6,0.792108,2,0.766451
7,0.552352,3,0.891954
8,0.164857,6,0.143009
9,0.528950,7,0.497675
